In [16]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [216]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
print(y.shape, tX.shape)

(250000,) (250000, 30)


In [245]:
with open(DATA_TRAIN_PATH, "r") as f:
    headers = f.readline().strip().split(',')[2:]

## Do your thing crazy machine learning thing here :) ...

In [405]:
from implementations import *

In [ ]:
# remove phi features
x, h = remove_phi_features(tX, headers)
print(x.shape)

In [407]:
# find highly correlated features and remove them
corr_features = find_correlated_features(x)

for k,v in corr_features.items():
    print("({:2d}){:25s} => ({:2d}){:25s}".format(k, h[k], v, h[v]))

# We can remove the features (15)PRI_lep_pt and (24)PRI_jet_all_pt
x = np.concatenate((x[:,:15], x[:,16:24]), axis=1)
h = h[:15] + h[16:24]
print(x.shape)

( 3)DER_pt_h                  => (24)PRI_jet_all_pt           
( 4)DER_deltaeta_jet_jet      => (24)PRI_jet_all_pt           
( 5)DER_mass_jet_jet          => (24)PRI_jet_all_pt           
( 6)DER_prodeta_jet_jet       => (24)PRI_jet_all_pt           
( 9)DER_sum_pt                => (24)PRI_jet_all_pt           
(10)DER_pt_ratio_lep_tau      => (15)PRI_lep_pt               
(12)DER_lep_eta_centrality    => (24)PRI_jet_all_pt           
(18)PRI_met_sumet             => (24)PRI_jet_all_pt           
(19)PRI_jet_num               => (24)PRI_jet_all_pt           
(20)PRI_jet_leading_pt        => (21)PRI_jet_leading_eta      
(22)PRI_jet_subleading_pt     => (24)PRI_jet_all_pt           
(23)PRI_jet_subleading_eta    => (24)PRI_jet_all_pt           
(250000, 23)


In [408]:
# split data in 3 categories according to the jet number
x0, y0, x1, y1, x23, y23 = split_data_jet_num(x, y, h)
assert x0.shape[0] + x1.shape[0] + x23.shape[0] == tX.shape[0]
print(x0.shape, y0.shape, x1.shape, y1.shape, x23.shape, y23.shape)

(99913, 22) (99913,) (77544, 22) (77544,) (72543, 22) (72543,)


In [409]:
# remove undefined features
x0 = remove_undefined_features(x0, h, True)
x1 = remove_undefined_features(x1, h, True)
x23 = remove_undefined_features(x23, h, True)
print(x0.shape, x1.shape, x23.shape)

(99913, 14) (77544, 16) (72543, 22)


In [410]:
# set remaining undefined values to the median of feature
x0 = set_undefined_to_median(x0)
x1 = set_undefined_to_median(x1)
x23 = set_undefined_to_median(x23)

In [411]:
# Normalisation
x0,_,_ = standardize(x0)
x1,_,_ = standardize(x1)
x23,_,_ = standardize(x23)

In [414]:
# train model
degrees = np.arange(6,9)
lambdas = np.logspace(-8, 0, 10)
threeModels = ThreeModels(x0,y0,x1,y1,x23,y23)
threeModels.train(degrees, lambdas, 5)

model1  Degree:  7 Lambda: 0.0021544347 Accuracy: 0.792, 
model0  Degree:  7 Lambda: 0.0000000100 Accuracy: 0.840, 
model23 Degree:  7 Lambda: 0.0021544347 Accuracy: 0.816, 


KeyboardInterrupt: 

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)